In [1]:
from scipy import constants
from astropy import units as u
import numpy as np 

# Toomre's Q parameter

In [94]:
def csound(T,m=2*1.00794*u.u,gamma=7/5):
    '''Calculates the speed of sound in a molecular gas at temperature T. By default assumes the gas is H2.
    m is the molecular mass and gamma the adiabatic index (7/5 for a diatomic gas and 5/3 for a monoatomic).
    Is mandatory to enter the units of the parameters m (u,kg,g,etc) and T (K).'''
    k = constants.k * u.kg * u.m**2 * u.s**(-2) * u.K**(-1) #Boltzmann constant 
    cs = (gamma*k*T/m.to(u.kg))**(1/2) #disk sound speed
    return cs

In [79]:
def Q(M_star,M_disk,T_disk,r_disk,c_sound):
    '''Calculates Toomre's parameter Q for a rotationally supported disk around a protostar (Kratter & Lodato 2016,
    Tobin 2016). Q < 1 means the disk is unstable (i.e. will fragment). Parameters are: M_star is the mass of 
    the protostar; M_disk, T_disk and r_disk are the mass, temperature and radius of the disk, respectively; 
    and c_sound is the speed of the sound in the gas. Is mandatory to enter the units of the parameters 
    M (solMass,kg,g,etc), T (K), r (au, km, m, etc) and c_sound (m/s).'''
    G = constants.G * u.m**3 * u.kg**(-1) * u.s**(-2) #Gravitational constant
    v = np.sqrt(G*M_star.to(u.kg)/r_disk.to(u.m)) #linear velocity at radius r
    w = v/r_disk.to(u.m) #Keplerian angular velocity
    H = c_sound/w # disk scale height
    Q = 2*M_star/M_disk * H/r_disk.to(u.m) 
    return Q

## Us

In [99]:
M_star = 0.41*u.solMass
M_disk = 0.14*u.solMass
T_disk=126*u.K #disk temperature
r_disk=150*u.au
c_sound=csound(T_disk)
Q(M_star,M_disk,T_disk,r_disk,c_sound)

<Quantity 3.20834049>

## Tobin (HOPS 370)

In [88]:
M_star = 2.5*u.solMass
M_disk = 0.06*u.solMass
T_disk=180*u.K #disk temperature
r_disk=50*u.au
c_sound=560*u.m*u.s**-1
#c_sound=csound(T_disk)
Q(M_star,M_disk,T_disk,r_disk,c_sound)

<Quantity 7.00694434>

# Mass from dust emission

In [144]:
def MassMA01(S,T,D,k_nu,f=100):
    '''Calculates the mass of gas from the dust emission using eq (1) in Motte & André (2001).Parameters are: S
    is the dust flux density; T is the dust temperature; D is the distance to the source; k_nu is the dust opacity
    at the frequency of the observation and f is the gas-to-dust ratio (by default 100). Is mandatory to enter the 
    units of the parameters S (mJy, Jy, etc), T (K), D (pc, km, m, etc), k_nu (cm^2/g).'''
    M = f*5.3e-3*(S.to(u.mJy)/(10*u.mJy))*(D.to(u.pc)/(140*u.pc))**2*(k_nu/(0.01*u.cm**2*u.g**-1))**(-1)*(T/(15*u.K))**-1
    return M*u.solMass

In [150]:
S = 1102*u.mJy
D = 300*u.pc
k_nu = 1.77*u.cm**2*u.g**-1 #en cm^2/g (del polvo)
f= 100 # gas-to-dust ratio
T=126 #temperature of the disk
ErT=23 #error of T
T_d = [i*u.K for i in [T,T-ErT,T+ErT]]
M=[MassMA01(S,t,D,k_nu) for t in T_d]
print(M)

[<Quantity 0.18038123 solMass>, <Quantity 0.22066054 solMass>, <Quantity 0.15253715 solMass>]


## M(dust) Eq(1) Tobin+2020 (hops370)

In [46]:
#Tobin (HOPS370)
S = 0.207e-30*u.J*u.cm**(-2)#en Jy
T_d=[180*u.K]
D=392*u.pc
k_nu=0.899*u.cm**2*u.g**(-1)

In [50]:
#us (SVS13)
S = 1.102e-30*u.J*u.cm**(-2)#en Jy
D=300*u.pc
T_d=[i*u.K for i in T_d]
k_nu=1.77*u.cm**2*u.g**(-1)

In [51]:
h=constants.h*u.J*u.s
l_onda=1.3*u.mm
c=constants.c*100*u.cm*u.s**-1
nu=c/l_onda.to(u.cm)
k=constants.k*u.J*u.K**-1
B=[2*h*nu**3/c**2 * 1/(np.exp(h*nu/(k*i))-1) for i in T_d]
M_d=[(D.to(u.cm))**2*S/(k_nu*i) for i in B] #de polvo
[i.to(u.M_sun)*100 for i in M_d] #de gas y polvo

[<Quantity 0.16831544 solMass>,
 <Quantity 0.13622668 solMass>,
 <Quantity 0.11440958 solMass>]

## Calculo N(H2)

In [ ]:
d_d=2 #diameter in arcsec
r_d=d_d/2*D*u.au #radio in AU 
A_d=np.pi*(r_d.to(u.cm))**2 #area of the disk
m_H2 = 2*1.00794*u.u #molecular mass (H_2)
N_H2_d=M_d*u.M_sun/(A_d*m_H2.to(u.M_sun)) 
print(N_H2_d)

## Calculo abundancia

In [ ]:
N_Sp_d = 3.9e14*u.cm**(-2) #column density of the species
Ab_Sp_d = N_Sp_d/N_H2_d #abundance of the species with respect to H2
print(Ab_Sp_d)

# From line emission

## M

In [ ]:
d_l=0.6 #diameter in arcsec
D=300 #distance in pc
N_Sp = 3.9e14*u.cm**(-2) #column density of the species
Frac_Ab_Sp = 1/45#fractional abundance of the species (13C/12C=1/45, 15N/14N=1/234 Crocket 2014 -> Tercero 2010)
Ab_Mol = 4.1e-9 #abundance of the molecule respect to H2 (HCN=3.8e-7 hot core, 4.1e-9 compact ridge, 2.8e-7 plateu, 3.3e-8 extendend ridge, Crocket 2014)
Ab_Sp = Frac_Ab_Sp * Ab_Mol #abundance of the species with respect to H2
r_l=d_l/2*D*u.au #radio in AU 
A_l=np.pi*(r_l.to(u.cm))**2 #area of the disk
m_H2 = 2*1.00794*u.u #molecular mass (H_2)
N_H2_l = N_Sp/Ab_Sp
M_H2_l = A_l*N_H2_l*m_H2.to(u.M_sun)
M_H2=M_H2_l*(r_d/r_l)**2
print(Ab_Sp)
print(N_H2_l)
print(M_H2_l)
print(M_H2)